# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:

cities_df = pd.read_csv("../output_data/cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [47]:
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]


In [48]:
fig = gmaps.figure()


heat_layer = gmaps.heatmap_layer(locations, weights=weight,dissipating=False, max_intensity=10, point_radius=1)


fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [14]:
formatted_cities = cities_df.dropna()

weather_fitted = formatted_cities.loc[(formatted_cities['Cloudiness'] > 0) & (formatted_cities['Wind Speed'] < 10)] 
weather_fitted_final = weather_fitted.loc[(weather_fitted['Max Temp'] > 65) & (weather_fitted['Max Temp'] <= 76)]
weather_fitted_final

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
9,9,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
59,59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70
86,86,nikolskoye,20,RU,1558378770,56,59.70,30.79,69.01,4.47
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
91,91,bella vista,62,AR,1558378771,80,-28.51,-59.05,75.66,7.29
97,97,katsuura,90,JP,1558378772,93,33.93,134.50,66.20,5.82
104,104,mbigou,70,GA,1558378773,94,-1.90,11.91,72.60,1.63
116,116,xam nua,100,LA,1558378775,98,20.42,104.04,67.38,2.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [15]:
hotel_df =  pd.DataFrame(weather_fitted_final, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
9,pushkinskiye gory,RU,57.02,28.91,
12,avarua,CK,-21.21,-159.78,
59,bontang,ID,0.12,117.47,
69,fortuna,ES,38.18,-1.13,
86,nikolskoye,RU,59.70,30.79,
87,ambilobe,MG,-13.19,49.05,
91,bella vista,AR,-28.51,-59.05,
97,katsuura,JP,33.93,134.50,
104,mbigou,GA,-1.90,11.91,
116,xam nua,LA,20.42,104.04,


In [16]:
# Print the name and address of the first Hotel
hotel_name = []

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

NameError: name 'params' is not defined

In [17]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()

ValueError: Length of values does not match length of index

In [18]:
hotel_df.to_csv('hotels.csv')

In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'narrowed_city_df' is not defined

In [20]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))